# <strong>End-to-End ML Project</strong>

There are 8 main steps for a ML project, also called the `ML checklist` in this book:

<ol>
  <li>Look at the big picture</li>
  
  <br>
  <li>Get the data</li><br>
  <li>Explore and visualize the data to gain insights</li><br>
  <li>Prepare the data for the ML algorithms</li><br>
  <li>Select a model and train it</li><br>
  <li>Fine-tune your model</li><br>
  <li>Present your solution</li><br>
  <li>Launch, monitor and maintain the system</li><br>
</ol>

Here we will analyze house prices in California in 1990. The data reflect block groups or districts (also called district, they are zones with 600 to 3000 people).

The goal is that the model predicts the median housing price in any district, given all the other metrics or features in the data set (`"housing.csv"`)



In [ ]:
import pandas as pd
import numpy as np

